# Model IO模块课程演练

In [1]:
## 导入依赖库

!pip install langchain

## 1. 什么是LCEL?

LangChain LangChain Expression Language(LCEL) 表达式语言；设计用来创建和组合链式结构的语法系统，链式操作
支持：并行处理、追踪、回调、批量、流式操作、同步等支持

## 2. Model

### 2.1 Model的分类

LLMs: LangChain 的核心组件。LangChain并不提供自己的LLMs，而是为与许多不同的LLMs（OpenAI、Cohere、Hugging Face等）进行交互提供了一个标准
接口。（类似于Completion，属于文本生成类模型的支持）

Chat Models: 语言模型的一种变体。虽然聊天模型在内部使用了语言模型，但它们提供的接口略有不同。与其暴露一个“输入文本，输出文本”的API不同，
它们提供了一个以“聊天消息”作为输入和输出的接口。(类似于Chat Completion，属于聊天会话类模型的支持)

### 2.2 LLMs

In [1]:
from langchain_openai import OpenAI

llm = OpenAI()

#老的使用方式
#llm("什么是机器学习？")
llm.invoke("奥巴马当了几年总统？")

'\n奥巴马担任美国总统的时间是2009年1月20日至2017年1月20日，共计8年。'

OpenAI API 地址：https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html#langchain.llms.openai.OpenAI

### 2.3 Chat Model

In [2]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# 设置模型角色，同时设置样例
messages = [SystemMessage(content="你是一个智能助手"),
 HumanMessage(content="第二十一届世界杯在哪儿举行的?"),
 AIMessage(content="在俄罗斯"),
 HumanMessage(content="冠军是哪个球队")]

In [3]:
chat_model.invoke(messages)

AIMessage(content='2018年世界杯的冠军是法国队。他们在决赛中击败了克罗地亚队。', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 67, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [5]:
chat_model.invoke(messages).content

'法国队在2018年世界杯中获得冠军。'

In [6]:
chat_model(messages)

AIMessage(content='2018年世界杯的冠军是法国队。')

ChatOpenAI API地址：https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html#langchain.chat_models.openai.ChatOpenAI

## 3.prompt

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或
进行对话。

提示模板（Prompt Templates）：参数化的模型输入

示例选择器（Example Selectors）：动态选择要包含在提示中的示例

### 3.1 Prompt Templates

#### 1. use PromptTemplate

In [4]:
## 可以动态传入参数
from langchain.prompts import PromptTemplate

# 定义提示词模板
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
# 传入参数，格式化提示词模板
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [5]:
## 也可以不传参数
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

In [8]:
llm.invoke(prompt_template.format())

'\n\nWhy was the math book sad?\n\nBecause it had too many problems.'

#### 2.use ChatPromptTemplate

In [6]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

chat_model.invoke(messages)

AIMessage(content='My name is Bob. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "你是一个智能助手，让用户变得更积极"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_model(chat_template.format_messages(text="我最近比较烦"))

AIMessage(content='我了解，人们常常会经历烦恼和困扰。但是，积极的态度和心态可以帮助我们应对困难和压力。让我来分享一些帮助你变得更积极的建议：\n\n1. 寻找支持：与朋友、家人或专业人士交流，分享你的烦恼，听取他们的建议和支持。\n\n2. 培养感恩之心：每天花一些时间思考你所拥有的，珍惜和感激这些东西。感恩之心可以帮助你更加积极地看待生活。\n\n3. 设定目标：制定一些具体的、可量化的目标，并制定实现这些目标的计划。追求目标可以给你带来成就感和动力。\n\n4. 培养健康的生活习惯：保持良好的饮食、充足的睡眠和适度的运动可以提升你的身体和心理健康。\n\n5. 积极思考：尝试改变消极的思维习惯，转而寻找积极的解释和观点。积极思考可以帮助你更好地应对挑战和困难。\n\n6. 寻找乐趣和爱好：找到一些你真正喜欢的活动或爱好，并定期参与其中。这样可以让你感到充实和快乐。\n\n7. 培养自我关爱：给自己一些时间和空间，关注自己的需求和感受。学会放松和照顾自己，这样你将更有能力面对困难。\n\n请记住，积极的态度需要时间和努力来培养。每天都尝试一些小的积极行为，逐渐改变你的思维方式。相信自己，你一定能够变得更积极和乐观！')

#### 3.Few-shot prompt templates

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples  = [
{
"question": "富兰克林·德拉诺·罗斯福和亚伯拉罕·林肯中哪个担任总统的时间更长？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：富兰克林·德拉诺·罗斯福担任了几年总统？
中间答案：富兰克林·德拉诺·罗斯福担任了12年的总统。
后续问题：亚伯拉罕·林肯则担任了几年总统？
中间答案：亚伯拉罕·林肯则担任了5年的总统。
因此最终答案是：富兰克林·德拉诺·罗斯福
"""
},
{
"question": "百度的创始人是何时出生的？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：百度的创始人是谁？
中间答案：百度的创始人是李彦宏。
后续问题：李彦宏是什么时候出生的？
中间答案：李彦宏于1968年11月17日出生。
因此最终答案是：1968年11月17日
"""
},
{
"question": "特雷西·麦克格雷迪和姚明，谁在NBA打的赛季多？",
"answer":
"""
这里需要后续问题吗：是。
后续问题：特雷西·麦克格雷迪在NBA打了几个赛季？
中间答案：特雷西·麦克格雷迪在NBA打了9个赛季。
后续问题：姚明在NBA打了几个赛季？
中间答案：姚明在NBA打了8个赛季。
因此最终答案是：特雷西·麦克格雷迪
"""
}
]

In [8]:
# 定义提示词模板
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], 
    template="Question: {question}\n{answer}")
# 使用第一个样例作为参数 并输出打印
print(example_prompt.format(**examples[0]))

Question: 富兰克林·德拉诺·罗斯福和亚伯拉罕·林肯中哪个担任总统的时间更长？

这里需要后续问题吗：是。
后续问题：富兰克林·德拉诺·罗斯福担任了几年总统？
中间答案：富兰克林·德拉诺·罗斯福担任了12年的总统。
后续问题：亚伯拉罕·林肯则担任了几年总统？
中间答案：亚伯拉罕·林肯则担任了5年的总统。
因此最终答案是：富兰克林·德拉诺·罗斯福



In [10]:
prompt = FewShotPromptTemplate(
    examples=examples,# 使用所有的样例
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="马云比马化腾大几岁？"))

Question: 富兰克林·德拉诺·罗斯福和亚伯拉罕·林肯中哪个担任总统的时间更长？

这里需要后续问题吗：是。
后续问题：富兰克林·德拉诺·罗斯福担任了几年总统？
中间答案：富兰克林·德拉诺·罗斯福担任了12年的总统。
后续问题：亚伯拉罕·林肯则担任了几年总统？
中间答案：亚伯拉罕·林肯则担任了5年的总统。
因此最终答案是：富兰克林·德拉诺·罗斯福


Question: 百度的创始人是何时出生的？

这里需要后续问题吗：是。
后续问题：百度的创始人是谁？
中间答案：百度的创始人是李彦宏。
后续问题：李彦宏是什么时候出生的？
中间答案：李彦宏于1968年11月17日出生。
因此最终答案是：1968年11月17日


Question: 特雷西·麦克格雷迪和姚明，谁在NBA打的赛季多？

这里需要后续问题吗：是。
后续问题：特雷西·麦克格雷迪在NBA打了几个赛季？
中间答案：特雷西·麦克格雷迪在NBA打了9个赛季。
后续问题：姚明在NBA打了几个赛季？
中间答案：姚明在NBA打了8个赛季。
因此最终答案是：特雷西·麦克格雷迪


Question: 马云比马化腾大几岁？


In [24]:
llm.invoke(prompt.format(input="特朗普的爸爸的女儿的兄弟是谁？"))

'\n\n这里需要后续问题吗：是。\n后续问题：特朗普的爸爸是谁？\n中间答案：特朗普的爸爸是弗雷德·特朗普。\n后续问题：弗雷德·特朗普的女儿是谁？\n中间答案：弗雷德·特朗普的女儿是玛丽·安娜·特朗普。\n后续问题：玛丽·安娜·特朗普的兄弟是谁？\n中间答案：玛丽·安娜·特朗普的兄弟是唐纳德·特朗普。\n因此最终答案是：唐纳德·特朗普'

### 3.2 Example selectors

#### 1.use Select by length

In [17]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,# 根据长度选择来选择不同的样例
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [18]:
## 输入较短，所以给的示例多
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [19]:
# 输入的内容长，給的示例少
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


## 4.Output parsers

语言模型输出文本。但是很多时候，你可能希望获得比纯文本更结构化的信息。这就是输出解析器的用处。

输出解析器是帮助结构化语言模型响应的类。一个输出解析器必须实现两个主要方法：

"获取格式指令"：返回一个包含语言模型输出应如何格式化的字符串的方法。
"解析"：接受一个字符串（假设是语言模型的响应）并将其解析为某种结构的方法。
然后还有一个可选的方法：

"带提示解析"：接受一个字符串（假设是语言模型的响应）和一个提示（假设是生成此响应的提示），并将其解析为某种结构。提示主要是在OutputParser
希望以某种方式重试或修复输出时提供的，它需要来自提示的信息来执行这些操作。

### 1.use List parser

当您想返回一个以逗号分隔的项目列表时，可以使用此输出解析器。

CommaSeparatedListOutputParser 解析逗号分隔列表的解析器

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

model = OpenAI()

parser = CommaSeparatedListOutputParser()
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)


## LCEL 链式表达式语言用法 (前一个的输入，作为后一个的输出)
chain = prompt | model | parser
chain.invoke({"subject": "ice cream flavors"})

['chocolate',
 'vanilla',
 'strawberry',
 'mint chocolate chip',
 'cookies and cream']